# Uncapacitated Facility Location Problem (UFL)

## Description
We want to open candidate facilities and assign each location to an open facility, so as
to minimise the total cost of opening facilities and assigning locations to open facilities.


## Set Up
### Sets
- N: set of locations, Choose some of them to be facilities

### Data
- $f_j$: fixed cost of using facility $j \in N$
- $c_{ij}$: cost of assigning loc i to facility j

### Variables
- $y_j \in \{0, 1\}$: 1 if location j is a facility
- $x_{ij} \in \{0, 1\}$: if location i is assigned to facility j

### Objective function
$$
\min \sum_{j \in N} y_j + \sum_{i \in N} \sum_{j \in N} c_{ij} x_{ij}
$$

### Constraints
Every location is assigned to a facility
$$
\sum_{j \in N} x_{ij} = 1 \forall i \in N
$$

A location can only be assigned to a facility that is open (note we do not use big M constraint here, big M is more computationaly intenstive since it is weaker constraint)
$$
x_{ij} \leq y_j \forall i \in N, j \in N
$$


In [4]:
import math
import random
import gurobipy as gp

random.seed(3)

nLocs = 300
I = range(nLocs)
J = range(nLocs)
F = [random.randint(10000,20000) for i in I]
C = [[random.randint(1000,2000) for j in J] for i in I]

m = gp.Model()

# Vars
Y = {j: m.addVar(vtype=gp.GRB.BINARY) for j in J}
X = {(i, j): m.addVar(vtype=gp.GRB.BINARY) for i in I for j in J}

# Constraints
one_facility = {
    i:
    m.addConstr(gp.quicksum(X[i, j] for j in J) == 1)
    for i in I
}

only_use_open_facilities = {
    (i, j):
    m.addConstr(X[i, j] <= Y[j])
    for (i, j) in X
}

m.setObjective(
    gp.quicksum(F[j] * Y[j] for j in J)
    + gp.quicksum(C[i][j] * X[i, j] for (i, j) in X)
)

m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 90300 rows, 90300 columns and 270000 nonzeros
Model fingerprint: 0x4804a68e
Variable types: 0 continuous, 90300 integer (90300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3284371.0000
Presolve time: 0.33s
Presolved: 90300 rows, 90300 columns, 270000 nonzeros
Variable types: 0 continuous, 90300 integer (90300 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 3.784392e+05, 39492 iterations, 3.51 seconds (12.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
